# Multi Dimensional Scaling (MDS)

Les méthodes de MDS consitent à projeter des données dans un espace de dimension réduite tout en conservant au mieux
une distance entre les données.

De nombreuses méthodes existent (voir par exemple http://en.wikipedia.org/wiki/Multidimensional_scaling, ou http://scikit-learn.org/stable/_downloads/plot_lle_digits.py) nous n'en présenterons ici que quelques une.

L'intérêt de ces méthodes est (au moins) double :
* il permet de représenter dansun espace de petite dimenstion des données a priri décrites dansun grand nombre de dimension
* il permet d'associer des axes à des données uniqueent décrite par une distance. Ceci permet de faire ensite une ACP dessus pour interpréter les données, faire des régressions, ou utiliser des algorithmes uniquement prévu pour le cas euclidien (comme les $k$-means par exemple).

## Données iris

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

current_palette = sns.color_palette()

In [ ]:
iris = sns.load_dataset('iris').drop(columns="species")
iris

Que représenter graphiquement ?

In [ ]:
sns.pairplot(iris)

plt.show()

On a toujours 3 espèces que l'on représentera en 3 clouleurs différentes. Les 50 premières sont de l'espèce *setosa*, les 50 suivantes de l'espèce *versicolor* et les 50 dernière de l'espèce *virginica*

**Attention** : ces 3 espèces sont des *meta* données : ce sont les botanistes qui ont répartis les iris en espèces, ce n'est pas inhérent aux données.

## Méthode 1 : les 2 premiers axes de l'ACP

On y reviendra, mais lACP est une méthode qui permet de conserver l'inertie du nuage de points.

Prendre les 2 premiers axes correspond à maximiser la projection des points.

### ACP de données non centrée/réduite

In [ ]:
from sklearn.decomposition import PCA 
import numpy as np
import pandas

In [ ]:
X = iris

pca = PCA()
pca.fit(X)

U = np.transpose(pca.components_) # vecteurs propres
I = pandas.DataFrame(np.transpose(pca.explained_variance_ratio_), columns=["pourcentage"])  # information véhiculée

C = pandas.DataFrame(X @ U, index=X.index) # nouvelles coordonnées

Pourcentage d'inertie (l'_information_) conservée :

In [ ]:
I

### On projette nos données sur les 2 premiers axes

In [ ]:
data = C.iloc[:, [0, 1]]

data

In [ ]:
data.dtypes

### On représente graphiquement le résultat

**bug** : avec les anciennes versions de seaborn, les noms de colonnes doivent être des `str` (sinon la designation d'une colonne comme axe (dans le sns.scatterplot ne fonctionne pas). Si cela vous arrive, renommez les colonnes en str (`data.columns = (str(x) for x in data.columns)`

On renomme donc les colonnes par des `str`.

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

sns.set()

current_palette = sns.color_palette()

In [ ]:
fig, ax = plt.subplots(figsize=(10, 10))

sns.scatterplot(x=0, 
                y=1, 
                data=data,
                legend=False,
                hue = [0] * 50 + [1] * 50 + [2] * 50,
                palette=current_palette[:3],
                ax=ax)


plt.show()

On peut même ajouter le label de cahque donnée :

In [ ]:
fig, ax = plt.subplots(figsize=(10, 10))

sns.scatterplot(x=0, 
                y=1, 
                data=data,
                legend=False,
                hue = [0] * 50 + [1] * 50 + [2] * 50,
                palette=current_palette[:3],
                ax=ax)

for index, row in data.iterrows():
    ax.annotate(str(index), (row[0], row[1]))

plt.show()

## Méthode 2 : le hasard

Cela peut sembler idiot, mais sur des données de grandes dimensions c'est (prouvé) assez efficace.

On utilise un module de sklearn https://scikit-learn.org/stable/modules/random_projection.html

### On choisi ici 2 nouveaux axes aléatoires

In [ ]:
from sklearn import random_projection

rp = random_projection.SparseRandomProjection(n_components=2, random_state=42)

### On projette nos données dessus

In [ ]:
iris_random = rp.fit_transform(iris)


iris_random = pandas.DataFrame(iris_random, index=iris.index)

iris_random

### On représente graphiquement le résultat

In [ ]:
data = iris_random

In [ ]:
data = iris_random

fig, ax = plt.subplots(figsize=(10, 10))

sns.scatterplot(x=0, 
                y=1, 
                data=data,
                legend=False,
                hue = [0] * 50 + [1] * 50 + [2] * 50,
                palette=current_palette[:3],
                ax=ax)
plt.show()

## Méthode 3 : le MDS classique

Nos données sont ici décrites par une distance (pas forcément euclidienne) et on veut trouver $k$ axes telle que cette distance corresponde à la distance euclienne de points sur ces $k$ axes. 

voir : https://fr.wikipedia.org/wiki/Positionnement_multidimensionnel


Ici nos iris sont décritent dans un espace à 4 dimensions. 

On va commencer par déterminer une distance entre nos iris. Nous allons utiliser la distance euclidienne mais il y en a plein d'autres de possible : 

https://scikit-learn.org/stable/modules/classes.html#sklearn-metrics-metrics

voir aussi https://towardsdatascience.com/3-basic-distance-measurement-in-text-mining-5852becff1d7 pour les 3 distances les plus utilisées (la dernière va vous surprendre)


### Distance euclidienne entre nos données

In [ ]:
from sklearn.metrics import euclidean_distances

d = euclidean_distances(iris)

d

In [ ]:
# c'est une liste de liste. Par exemple distance entre l'élément 0 et l'élément 42
d[0][42]

In [ ]:
# ou au format numpy

d[0, 42]

On cherche ensuite 2 dimensions pour les quelles cette distance serait bien conservée

https://scikit-learn.org/stable/modules/generated/sklearn.manifold.MDS.html#sklearn.manifold.MDS

### On trouve 2 axes permettant de recréer la distance de façon approchée

In [ ]:
from sklearn import manifold

In [ ]:
mds = manifold.MDS(n_components=2, max_iter=3000,
                   dissimilarity="precomputed", n_jobs=1, n_init=5)

pos = mds.fit(d).embedding_
pos

### On représente graphiquement le résultat

In [ ]:
data = pandas.DataFrame(pos)

fig, ax = plt.subplots(figsize=(10, 10))

sns.scatterplot(x=0, 
                y=1, 
                data=data,
                legend=False,
                hue = [0] * 50 + [1] * 50 + [2] * 50,
                palette=current_palette[:3],
                ax=ax)
plt.show()

## Méthode 4 : isomap

On essaie non pas de conserver toutes les distances mais seulement les $k$ plus proches.

https://scikit-learn.org/stable/modules/generated/sklearn.manifold.Isomap.html#sklearn.manifold.Isomap

### On trouve 2 axes permettant de recréer la distance des 10 plus proches voisins de façon approchée

In [ ]:
mds = manifold.Isomap(n_neighbors=10,
                      n_components=2, max_iter=3000,                      
                      metric="precomputed")

pos = mds.fit(d).embedding_
pos

### On représente graphiquement le résultat


In [ ]:
data = pandas.DataFrame(pos)

fig, ax = plt.subplots(figsize=(10, 10))

sns.scatterplot(x=0, 
                y=1, 
                data=data,
                legend=False,
                hue = [0] * 50 + [1] * 50 + [2] * 50,
                palette=current_palette[:3],
                ax=ax)
plt.show()